 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/Test_step4.ipynb#模型测试" data-toc-modified-id="模型测试-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>模型测试</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/Test_step4.ipynb#导入工具包，并选择GPU卡" data-toc-modified-id="导入工具包，并选择GPU卡-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>导入工具包，并选择GPU卡</a></span></li><li><span><a href="http://localhost:8888/notebooks/Test_step4.ipynb#默认参数设置" data-toc-modified-id="默认参数设置-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>默认参数设置</a></span></li><li><span><a href="http://localhost:8888/notebooks/Test_step4.ipynb#导入模型以及所需的信息" data-toc-modified-id="导入模型以及所需的信息-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>导入模型以及所需的信息</a></span></li></ul></li></ul></div>

# 模型测试
这是Image_Caption第四个运行的代码,就是用第三步已经训练好的模型对测试数据进行测试并输出最终的结果并进行提交.

## 导入工具包，并选择GPU卡

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import numpy as np

import time
import os
from six.moves import cPickle

import opts
import models
from dataloader import *
# from dataloaderraw import *
import eval_utils
import argparse
import misc.utils as utils
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

## 默认参数设置
- model: 第三步训练得到的模型
- infos_path: 与model对应的数据的信息,包括词典等信息
- cnn_model:对图像进行预处理的模型，默认的是resnet152
- image_folder: 测试图片的位置
- split: 因为此处是为了测试使用,所以需要设置为test,前面的是train和val
- 其他的特征都和网络结构有关,请自行理解.

In [ ]:
class Opt():
    def __init__(self):
        self.model = './log_st/model-best_attention.pth'
        self.cnn_model = 'resnet152' 
        self.infos_path = './log_st/infos_attention.pkl'
        
        self.batch_size = 20 
        self.num_images = -1 
        self.language_eval = 0  
        self.dump_images = 1 
        self.dump_json =1
        self.dump_path = 0
        
        
        self.sample_max = 1 #
        
        self.beam_size = 2
        self.temperature = 1.0
        self.image_folder = '/home/jiangqy/EXP/competition/data/captions/ai_challenger_caption_test1_20170923/caption_test1_images_20170923'
        self.image_root  = ''
        self.input_fc_dir = ''
        self.input_att_dir  = ''

        self.input_label_h5 = ''
        self.input_json = ''
        self.split = 'test'

        self.coco_json = ''
        self.id = ''

In [ ]:
opt = Opt() 
opt.infos_path = './log_st/infos_-best_attention.pkl'

## 导入模型以及所需的信息

In [ ]:
# Load infos
with open(opt.infos_path) as f:
    infos = cPickle.load(f)

In [ ]:
# override and collect parameters
if len(opt.input_fc_dir) == 0:
    opt.input_fc_h5 = infos['opt'].input_fc_h5
    opt.input_att_h5 = infos['opt'].input_att_h5
    opt.input_label_h5 = infos['opt'].input_label_h5 
if len(opt.input_json) == 0:
    opt.input_json = infos['opt'].input_json
if opt.batch_size == 0:
    opt.batch_size = infos['opt'].batch_size
if len(opt.id) == 0:
    opt.id = infos['opt'].id
ignore = ["id", "batch_size", "beam_size", "start_from", "language_eval"]

In [ ]:
for k in vars(infos['opt']).keys():
    if k not in ignore:
        if k in vars(opt):
            assert vars(opt)[k] == vars(infos['opt'])[k], k + ' option not consistent'
        else:
            vars(opt).update({k: vars(infos['opt'])[k]}) # copy over options from model 
vocab = infos['vocab'] # ix -> word mapping

In [ ]:
from dataloaderraw import *
if len(opt.image_folder) == 0:
    loader = DataLoader(opt)
else:
    loader = DataLoaderRaw({'folder_path': opt.image_folder, 
                            'coco_json': opt.coco_json,
                            'batch_size': opt.batch_size,
                            'cnn_model': opt.cnn_model})
loader.ix_to_word = infos['vocab']

In [ ]:
# Setup the model
model = models.setup(opt)
# opt.batch_size = 20
model.load_state_dict(torch.load(opt.model))
model.cuda()
model.eval()
crit = utils.LanguageModelCriterion()

** beam_size ** 的设置相对比较重要,show_and_tell中贪心和设置为5线上成绩相差了接近2个点.

In [ ]:
opt.beam_size = 5 
loss, split_predictions, lang_stats = eval_utils.eval_split_test(model, crit, loader, vars(opt))

In [ ]:
for item in split_predictions:
    item['caption'] = (item['caption']).replace(' ','')
    
json.dump(split_predictions, open('./res/res_attention_new_8.json', 'w'))